In [24]:
# === Block 1: Directory Selection ===
# Run this first to select the directory containing your Excel files

import tkinter as tk
from tkinter import filedialog
import os

def select_directory():
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    directory_path = filedialog.askdirectory(
        title="Select Directory Containing Excel Files"
    )
    root.destroy()
    if not directory_path:
        raise FileNotFoundError("No directory selected.")
    return directory_path

# Run the directory selector
base_directory = select_directory()
print("Selected directory:", base_directory)

Selected directory: C:/Users/Instrument_Room/OneDrive - UCB-O365 (1)/Courses/Year 1/Fall Semester Aug-Dec 2020/CHEN 5840 - Independent Study/Hydrogels/Rheometer experiments/Fall Semester 2026


In [25]:
# === Block 2 Fixed: Find CSV Files and Calculate Global Y-Max ===
import os
import pandas as pd
import numpy as np

# Convert base_directory to short path format to handle OneDrive issues
try:
    import win32api
    base_directory_short = win32api.GetShortPathName(base_directory)
    print(f"Converted to short path: {base_directory_short}\n")
    base_directory = base_directory_short
except:
    print("Note: Could not convert to short path format")
    print("If you get file not found errors, manually set base_directory to short path format\n")

def find_csv_files(directory):
    """Recursively find all CSV files in directory and subdirectories, excluding 'old' folders."""
    csv_files = []
    for root, dirs, files in os.walk(directory):
        # Skip 'old' directories
        if 'old' in [d.lower() for d in root.split(os.sep)]:
            continue
            
        for file in files:
            if file.endswith('.csv') and not file.startswith('~$'):
                csv_files.append(os.path.join(root, file))
    return sorted(csv_files)

def detect_cycle_in_filename(filename):
    """Detect which cycle type is in the filename."""
    lower = filename.lower()
    
    if '40 cycles' in lower or '40cycles' in lower:
        return '40 cycles', '3.6%'
    elif '28 cycles' in lower or '28cycles' in lower:
        return '28 cycles', '2.5%'
    elif '0 cycles' in lower or '0cycles' in lower:
        return '0 cycles', '0%'
    else:
        return None, None

def detect_focus_distance_filename(filename):
    """Detect whether file is Focus (F) or Distance (D) measurement."""
    basename = os.path.basename(filename).upper()
    
    if basename.startswith('F_'):
        return 'Focus'
    elif basename.startswith('D_'):
        return 'Distance'
    
    return None

def extract_gelatin_concentration(file_path):
    """Extract gelatin concentration from file path."""
    import re
    match = re.search(r'(\d+)%\s*gelatin', file_path, re.IGNORECASE)
    if match:
        return f"{match.group(1)}%"
    return None

def calculate_global_ymax_from_csv(csv_files, freq_window=(0.3, 1.0)):
    """Calculate the maximum AVERAGE storage modulus across all CSV files."""
    global_max_avg = 0
    successful_reads = 0
    failed_reads = 0
    
    print(f"Processing {len(csv_files)} CSV files...")
    print(f"Calculating average modulus in frequency window: {freq_window[0]}-{freq_window[1]} rad/s\n")
    
    for csv_path in csv_files:
        filename = os.path.basename(csv_path)
        lower = filename.lower()
        
        # Check if file contains a cycle type
        cycle_type, duty_cycle = detect_cycle_in_filename(filename)
        has_cycle = cycle_type is not None
        
        # Check for particles
        has_particle = any(p in lower for p in ['dbpc hmsn', 'dbpc_hmsn', 'msn', 'pbs'])
        
        # Detect focus/distance
        focus_type = detect_focus_distance_filename(filename)
        
        if has_cycle and has_particle:
            try:
                # Skip first 53 rows (metadata + headers + units)
                # Data starts at line 54
                df = pd.read_csv(csv_path, skiprows=53, header=None)
                
                # Column 0 = Angular frequency
                # Column 6 = Storage modulus
                if len(df) > 0 and len(df.columns) > 6:
                    # Get frequency and storage modulus data
                    freq_data = df.iloc[:, 0]
                    modulus_data = df.iloc[:, 6]
                    
                    # Create a clean dataframe
                    data = pd.DataFrame({
                        'Angular Frequency': pd.to_numeric(freq_data, errors='coerce'),
                        'Storage Modulus': pd.to_numeric(modulus_data, errors='coerce')
                    }).dropna()
                    
                    if not data.empty:
                        # Filter for frequency window
                        mask = (data['Angular Frequency'] >= freq_window[0]) & \
                               (data['Angular Frequency'] <= freq_window[1])
                        window_data = data[mask]
                        
                        if not window_data.empty:
                            # Calculate average in window
                            avg_modulus = window_data['Storage Modulus'].mean()
                            global_max_avg = max(global_max_avg, avg_modulus)
                            
                            focus_label = f" [{focus_type}]" if focus_type else ""
                            print(f"    ✓ {filename}: avg = {avg_modulus:.1f} Pa ({cycle_type}){focus_label}")
                            successful_reads += 1
                        else:
                            print(f"    ⚠ {filename}: no data in frequency window")
                            failed_reads += 1
                    else:
                        print(f"    ⚠ {filename}: no valid numeric data")
                        failed_reads += 1
                else:
                    print(f"    ⚠ {filename}: insufficient data")
                    failed_reads += 1
                    
            except Exception as e:
                print(f"    ✗ {filename}: Error - {str(e)[:100]}")
                failed_reads += 1
    
    print(f"\n{'='*60}")
    print(f"SUMMARY:")
    print(f"  Successfully read: {successful_reads}/{len(csv_files)} files")
    print(f"  Failed to read: {failed_reads}/{len(csv_files)} files")
    print(f"  Global Y-max (based on averages): {global_max_avg:.1f} Pa")
    print(f"{'='*60}\n")
    
    return global_max_avg

# Find all CSV files
csv_files = find_csv_files(base_directory)
print(f"\nFound {len(csv_files)} CSV files:")

# Show first 10 files as preview
for f in csv_files[:10]:
    print(f"  - {f}")
if len(csv_files) > 10:
    print(f"  ... and {len(csv_files) - 10} more")

print("\n")

# Calculate global maximum based on averages
if csv_files:
    global_ymax = calculate_global_ymax_from_csv(csv_files)
    print(f"\nGlobal Y-max to use for all plots: {global_ymax:.1f} Pa")
else:
    print("\n⚠ WARNING: No CSV files found!")
    print("Check that your base_directory is correct and contains CSV files")
    global_ymax = 0

Converted to short path: C:/Users/INSTRU~1/ONEDRI~2/Courses/YEAR1~1/FALLSE~1/CHEN58~1/HYDROG~1/RHEOME~1/FALLSE~1


Found 108 CSV files:
  - C:/Users/INSTRU~1/ONEDRI~2/Courses/YEAR1~1/FALLSE~1/CHEN58~1/HYDROG~1/RHEOME~1/FALLSE~1\06OCT25\2% gelatin\1X PBS\0 cycles\D_1X PBS_0 cycles_1.csv
  - C:/Users/INSTRU~1/ONEDRI~2/Courses/YEAR1~1/FALLSE~1/CHEN58~1/HYDROG~1/RHEOME~1/FALLSE~1\06OCT25\2% gelatin\1X PBS\0 cycles\D_1X PBS_0 cycles_2.csv
  - C:/Users/INSTRU~1/ONEDRI~2/Courses/YEAR1~1/FALLSE~1/CHEN58~1/HYDROG~1/RHEOME~1/FALLSE~1\06OCT25\2% gelatin\1X PBS\0 cycles\D_1X PBS_0 cycles_3.csv
  - C:/Users/INSTRU~1/ONEDRI~2/Courses/YEAR1~1/FALLSE~1/CHEN58~1/HYDROG~1/RHEOME~1/FALLSE~1\06OCT25\2% gelatin\1X PBS\0 cycles\F_1X PBS_0 cycles_1.csv
  - C:/Users/INSTRU~1/ONEDRI~2/Courses/YEAR1~1/FALLSE~1/CHEN58~1/HYDROG~1/RHEOME~1/FALLSE~1\06OCT25\2% gelatin\1X PBS\0 cycles\F_1X PBS_0 cycles_2.csv
  - C:/Users/INSTRU~1/ONEDRI~2/Courses/YEAR1~1/FALLSE~1/CHEN58~1/HYDROG~1/RHEOME~1/FALLSE~1\06OCT25\2% gelati

In [33]:
# === Block 3: Plot All CSV Files Organized by Gelatin Concentration + Focus/Distance ===
# Small-panel (3.77" x 2.50") figures with legible fonts, 600 dpi

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Rectangle
from matplotlib.legend_handler import HandlerBase
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter, AutoMinorLocator
import matplotlib as mpl
import colorsys
from scipy.stats import ttest_ind, ttest_rel
from matplotlib.colors import to_rgb, to_hex
import re

# ===== Small-panel & style settings =====
PANEL_W_IN, PANEL_H_IN = 3.77, 2.50
EXPORT_DPI = 600

AXIS_FS = 11
TICK_FS = 10
LEGEND_FS = 9

ERR_KW = dict(elinewidth=0.6, capsize=3, capthick=0.6, ecolor="black")

HATCH_LW_BARS = 0.4
HATCH_LW_LEG  = 0.6
mpl.rcParams['hatch.linewidth'] = HATCH_LW_BARS

# ===== Pattern definitions (shared across functions) =====
GELATIN_PATTERNS = {
    '2%': '',
    '4%': '//',
    '8%': 'xx'
}

class HandlerLineWithShade(HandlerBase):
    def create_artists(self, legend, orig_handle,
                      xdescent, ydescent, width, height, fontsize,
                      trans):
        color = orig_handle.get_color()
        rect = Rectangle((xdescent, ydescent),
                         width, height,
                         facecolor=color,
                         edgecolor='none',
                         alpha=0.2,
                         transform=trans)
        line = Line2D([xdescent, xdescent + width],
                      [ydescent + height/2, ydescent + height/2],
                      color=color,
                      linewidth=3,
                      transform=trans)
        return [rect, line]

def small_panel_ax():
    fig, ax = plt.subplots(figsize=(PANEL_W_IN, PANEL_H_IN))
    ax.tick_params(axis='both', which='major',
                   labelsize=TICK_FS, length=3, width=0.8, direction='out')
    ax.tick_params(axis='y', which='minor',
                   length=2, width=0.5, direction='out')
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    for sp in ax.spines.values():
        sp.set_linewidth(0.8)
    return fig, ax

def small_legend(ax, *args, **kwargs):
    leg = ax.legend(*args, fontsize=LEGEND_FS, title_fontsize=LEGEND_FS,
                    frameon=True, fancybox=True, framealpha=1.0, **kwargs)
    fr = leg.get_frame()
    fr.set_edgecolor("#D0D0D0")
    fr.set_linewidth(0.8)
    return leg

def adjust_lightness(hex_color, factor):
    """Lighten (factor > 1) or darken (factor < 1) a hex color."""
    r, g, b = to_rgb(hex_color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    l = max(0, min(1, l * factor))
    r2, g2, b2 = colorsys.hls_to_rgb(h, l, s)
    return to_hex((r2, g2, b2))

def extract_gelatin_concentration(file_path):
    """Extract gelatin concentration from file path."""
    match = re.search(r'(\d+)%\s*gelatin', file_path, re.IGNORECASE)
    if match:
        return f"{match.group(1)}%"
    return None

def detect_cycle_in_filename(filename):
    """Detect which cycle type is in the filename."""
    lower = filename.lower()
    if '40 cycles' in lower or '40cycles' in lower:
        return '40 cycles', '3.6%'
    elif '28 cycles' in lower or '28cycles' in lower:
        return '28 cycles', '2.5%'
    elif '0 cycles' in lower or '0cycles' in lower:
        return '0 cycles', '0%'
    else:
        return None, None

def detect_focus_distance_filename(filename):
    """Detect whether file is Focus (F) or Distance (D) measurement."""
    basename = os.path.basename(filename).upper()
    if basename.startswith('F_'):
        return 'Focus'
    elif basename.startswith('D_'):
        return 'Distance'
    return None

def find_common_base_directory(csv_files):
    """Find the common parent directory for all CSV files."""
    if not csv_files:
        return os.getcwd()
    
    # Get all directory paths
    dirs = [os.path.dirname(os.path.abspath(f)) for f in csv_files]
    
    # Find common path
    common = os.path.commonpath(dirs)
    
    return common

def plot_gelatin_organized_data(csv_files, global_ymax_storage, global_ymax_loss, base_directory, add_dots=False, output_suffix=""):
    """Plot data organized by gelatin concentration for both storage and loss modulus."""
    
    # --- Settings ---
    start_row = 0
    freq_window = (0.3, 1.0)
    
    cycle_map = {
        '0 cycles': '0%',
        '28 cycles': '2.5%',
        '40 cycles': '3.6%'
    }
    
    particle_colors = {
        'DBPC HMSN': '#D55E00',
        'MSN': '#009E73',
        '1X PBS': '#0072B2',
    }
    
    shade_map = {
        '0%': 1.3,
        '2.5%': 1.0,
        '3.6%': 0.7
    }
    
    # Add dots to patterns if this is for Distance plots
    gelatin_patterns = {k: v + ('...' if add_dots else '') for k, v in GELATIN_PATTERNS.items()}

    # Collect all data
    all_data = {}
    file_data = {}
    
    for csv_path in csv_files:
        gelatin_conc = extract_gelatin_concentration(csv_path)
        if not gelatin_conc:
            continue
            
        if gelatin_conc not in all_data:
            all_data[gelatin_conc] = {
                p: {dc: {'storage': [], 'loss': []} for dc in cycle_map.values()} 
                for p in particle_colors
            }
            file_data[gelatin_conc] = {
                p: {dc: {'Focus': [], 'Distance': []} for dc in cycle_map.values()}
                for p in particle_colors
            }
        
        try:
            filename = os.path.basename(csv_path)
            lower = filename.lower()
            
            cycle_type, dc = detect_cycle_in_filename(filename)
            if not (cycle_type and dc):
                continue
            
            if 'dbpc hmsn' in lower or 'dbpc_hmsn' in lower or 'dbpchmsn' in lower:
                particle = 'DBPC HMSN'
            elif 'msn' in lower and 'hmsn' not in lower:
                particle = 'MSN'
            elif '1x pbs' in lower or '1xpbs' in lower or 'pbs' in lower:
                particle = '1X PBS'
            else:
                continue
            
            focus_dist = detect_focus_distance_filename(filename)
            if focus_dist:
                file_data[gelatin_conc][particle][dc][focus_dist].append(csv_path)
            
            df = pd.read_csv(csv_path, skiprows=53, header=None)
            
            storage_data = df.iloc[start_row:, [0, 6]].dropna()
            storage_data.columns = ['Angular Frequency', 'Storage Modulus']
            storage_data = storage_data.apply(pd.to_numeric, errors='coerce').dropna()
            
            loss_data = df.iloc[start_row:, [0, 7]].dropna()
            loss_data.columns = ['Angular Frequency', 'Loss Modulus']
            loss_data = loss_data.apply(pd.to_numeric, errors='coerce').dropna()
            
            if not storage_data.empty:
                all_data[gelatin_conc][particle][dc]['storage'].append(storage_data)
            if not loss_data.empty:
                all_data[gelatin_conc][particle][dc]['loss'].append(loss_data)
                
        except Exception as e:
            print(f"Error processing {csv_path}: {e}")
            continue
    
    output_dir = os.path.join(base_directory, f"plots_by_gelatin{output_suffix}")
    os.makedirs(output_dir, exist_ok=True)
    
    plots_created = []
    t_test_results = []
    paired_t_test_results = []
    
    sorted_gelatin = sorted(all_data.keys(), key=lambda x: int(x.rstrip('%')))
    
    # Bar plots for each gelatin concentration
    for gelatin_conc in sorted_gelatin:
        gelatin_data = all_data[gelatin_conc]
        
        for modulus_type in ['storage', 'loss']:
            modulus_label = 'Storage' if modulus_type == 'storage' else 'Loss'
            modulus_symbol = "G'" if modulus_type == 'storage' else 'G"'
            global_ymax = global_ymax_storage if modulus_type == 'storage' else global_ymax_loss
            
            particles_with_data = [p for p in particle_colors 
                                  if any(gelatin_data[p][dc][modulus_type] for dc in cycle_map.values())]
            
            if not particles_with_data:
                continue
            
            fig, ax = small_panel_ax()
            
            sorted_dcs = sorted([dc for dc in cycle_map.values() if 
                               any(gelatin_data[p][dc][modulus_type] for p in particles_with_data)],
                               key=lambda x: float(x.rstrip('%')))
            
            x = np.arange(len(particles_with_data))
            width = 0.6
            offsets = np.linspace(-width, width, len(sorted_dcs))
            
            for i, dc in enumerate(sorted_dcs):
                means = []
                stds = []
                
                for particle in particles_with_data:
                    dfs = gelatin_data[particle][dc][modulus_type]
                    if dfs:
                        vals = []
                        for df in dfs:
                            mask = (df['Angular Frequency'] >= freq_window[0]) & (df['Angular Frequency'] <= freq_window[1])
                            mean_val = df.loc[mask, f'{modulus_label} Modulus'].mean()
                            if not np.isnan(mean_val):
                                vals.append(mean_val)
                        
                        means.append(np.mean(vals) if vals else 0)
                        stds.append(np.std(vals) if vals else 0)
                    else:
                        means.append(0)
                        stds.append(0)
                
                bars = ax.bar(x + offsets[i], means, width, yerr=stds, 
                             label=dc, error_kw=ERR_KW)
                
                for j, bar in enumerate(bars):
                    particle = particles_with_data[j]
                    base_color = particle_colors[particle]
                    bar_color = adjust_lightness(base_color, shade_map[dc])
                    bar.set_color(bar_color)
                    bar.set_hatch(gelatin_patterns[gelatin_conc])
                    bar.set_edgecolor('gray')
                    bar.set_linewidth(0.5)
            
            # Independent t-tests between duty cycles
            for p_idx, particle in enumerate(particles_with_data):
                for i in range(len(sorted_dcs) - 1):
                    dc1, dc2 = sorted_dcs[i], sorted_dcs[i+1]
                    
                    vals1 = []
                    for df in gelatin_data[particle][dc1][modulus_type]:
                        mask = (df['Angular Frequency'] >= freq_window[0]) & (df['Angular Frequency'] <= freq_window[1])
                        mean_val = df.loc[mask, f'{modulus_label} Modulus'].mean()
                        if not np.isnan(mean_val):
                            vals1.append(mean_val)
                    
                    vals2 = []
                    for df in gelatin_data[particle][dc2][modulus_type]:
                        mask = (df['Angular Frequency'] >= freq_window[0]) & (df['Angular Frequency'] <= freq_window[1])
                        mean_val = df.loc[mask, f'{modulus_label} Modulus'].mean()
                        if not np.isnan(mean_val):
                            vals2.append(mean_val)
                    
                    if len(vals1) >= 2 and len(vals2) >= 2:
                        t_stat, p_val_two = ttest_ind(vals1, vals2)
                        p_val_one = p_val_two / 2
                        
                        sig_two = "***" if p_val_two < 0.001 else "**" if p_val_two < 0.01 else "*" if p_val_two < 0.05 else "ns"
                        sig_one = "***" if p_val_one < 0.001 else "**" if p_val_one < 0.01 else "*" if p_val_one < 0.05 else "ns"
                        
                        result_str = (f"{gelatin_conc} Gelatin - {particle} {modulus_label}: "
                                    f"{dc1} vs {dc2} | "
                                    f"Two-tailed: p={p_val_two:.4f} {sig_two}, "
                                    f"One-tailed: p={p_val_one:.4f} {sig_one}")
                        t_test_results.append(result_str)
            
            ax.set_xlabel('Particle Type', fontsize=AXIS_FS)
            ax.set_ylabel(f'{modulus_label} Modulus ({modulus_symbol})\n(Pa)', fontsize=AXIS_FS)
            ax.set_xticks(x)
            ax.set_xticklabels(particles_with_data, fontsize=TICK_FS)
            ax.set_ylim(0, global_ymax * 1.1)
            ax.ticklabel_format(style='plain', axis='y', useOffset=False)
            ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x):,}'))
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            
            gray_map = {'0%': '#E0E0E0', '2.5%': '#A0A0A0', '3.6%': '#606060'}
            
            with mpl.rc_context({'hatch.linewidth': HATCH_LW_LEG}):
                handles = [Patch(facecolor=gray_map.get(dc, 'gray'), 
                                 edgecolor='gray', 
                                 linewidth=0.5,
                                 alpha=0.8 if dc == '2.5%' else 1.0,
                                 hatch=gelatin_patterns[gelatin_conc])
                           for dc in sorted_dcs]
            small_legend(ax, handles=handles,
                         labels=sorted_dcs,
                         title="Duty Cycle", 
                         bbox_to_anchor=(1.00, 1), 
                         loc='upper left')
            
            plt.tight_layout()
            
            filename = f"gelatin_{gelatin_conc}_{modulus_type}_comparison.png"
            filepath = os.path.join(output_dir, filename)
            plt.savefig(filepath, dpi=EXPORT_DPI, bbox_inches='tight')
            plt.close()
            plots_created.append(filename)
    
    # Ratio bar plots
    for gelatin_conc in sorted_gelatin:
        gelatin_data = all_data[gelatin_conc]
        
        particles_with_data = [p for p in particle_colors 
                              if any(gelatin_data[p][dc]['storage'] and gelatin_data[p][dc]['loss'] 
                                    for dc in cycle_map.values())]
        
        if not particles_with_data:
            continue
        
        fig, ax = small_panel_ax()
        
        sorted_dcs = sorted([dc for dc in cycle_map.values() if 
                           any(gelatin_data[p][dc]['storage'] and gelatin_data[p][dc]['loss'] 
                              for p in particles_with_data)],
                           key=lambda x: float(x.rstrip('%')))
        
        x = np.arange(len(particles_with_data))
        width = 0.6
        offsets = np.linspace(-width, width, len(sorted_dcs))
        
        for i, dc in enumerate(sorted_dcs):
            means = []
            stds = []
            
            for particle in particles_with_data:
                storage_list = gelatin_data[particle][dc]['storage']
                loss_list = gelatin_data[particle][dc]['loss']
                
                if storage_list and loss_list:
                    ratios = []
                    for s_df, l_df in zip(storage_list, loss_list):
                        merged = pd.merge(s_df, l_df, on='Angular Frequency', suffixes=('_s', '_l'))
                        mask = (merged['Angular Frequency'] >= freq_window[0]) & (merged['Angular Frequency'] <= freq_window[1])
                        if mask.any():
                            ratio = (merged.loc[mask, 'Storage Modulus'].mean() / 
                                   merged.loc[mask, 'Loss Modulus'].mean())
                            if not np.isnan(ratio):
                                ratios.append(ratio)
                    
                    means.append(np.mean(ratios) if ratios else 0)
                    stds.append(np.std(ratios) if ratios else 0)
                else:
                    means.append(0)
                    stds.append(0)
            
            bars = ax.bar(x + offsets[i], means, width, yerr=stds,
                         label=dc, error_kw=ERR_KW)
            
            for j, bar in enumerate(bars):
                particle = particles_with_data[j]
                base_color = particle_colors[particle]
                bar_color = adjust_lightness(base_color, shade_map[dc])
                bar.set_color(bar_color)
                bar.set_hatch(gelatin_patterns[gelatin_conc])
                bar.set_edgecolor('gray')
                bar.set_linewidth(0.5)
        
        ax.set_xlabel('Particle Type', fontsize=AXIS_FS)
        ax.set_ylabel("G'/G'' Ratio", fontsize=AXIS_FS)
        ax.set_xticks(x)
        ax.set_xticklabels(particles_with_data, fontsize=TICK_FS)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        gray_map = {'0%': '#E0E0E0', '2.5%': '#A0A0A0', '3.6%': '#606060'}
        
        with mpl.rc_context({'hatch.linewidth': HATCH_LW_LEG}):
            handles = [Patch(facecolor=gray_map.get(dc, 'gray'), 
                             edgecolor='gray', 
                             linewidth=0.5,
                             alpha=0.8 if dc == '2.5%' else 1.0,
                             hatch=gelatin_patterns.get(gelatin_conc, ''))
                       for dc in sorted_dcs]
        small_legend(ax, handles=handles,
                     labels=sorted_dcs,
                     title="Duty Cycle", 
                     bbox_to_anchor=(1.00, 1), 
                     loc='upper left')
        
        plt.tight_layout()
        
        filename = f"gelatin_{gelatin_conc}_ratio_comparison.png"
        filepath = os.path.join(output_dir, filename)
        plt.savefig(filepath, dpi=EXPORT_DPI, bbox_inches='tight')
        plt.close()
        plots_created.append(filename)
    
    # Ratio line plots
    for gelatin_conc in sorted_gelatin:
        gelatin_data = all_data[gelatin_conc]
        
        for particle in particle_colors:
            has_data = any(gelatin_data[particle][dc]['storage'] and 
                          gelatin_data[particle][dc]['loss'] for dc in cycle_map.values())
            if not has_data:
                continue
            
            base_color = particle_colors[particle]
            fig, ax = small_panel_ax()
            
            legend_handles = []
            legend_labels = []
            
            sorted_dcs = sorted([dc for dc in cycle_map.values() if 
                               gelatin_data[particle][dc]['storage'] and 
                               gelatin_data[particle][dc]['loss']], 
                               key=lambda x: float(x.rstrip('%')))
            
            for dc in sorted_dcs:
                storage_list = gelatin_data[particle][dc]['storage']
                loss_list = gelatin_data[particle][dc]['loss']
                if not storage_list or not loss_list:
                    continue
                    
                color = adjust_lightness(base_color, shade_map[dc])
                
                all_ratios = []
                for s_df, l_df in zip(storage_list, loss_list):
                    merged = pd.merge(s_df, l_df, on='Angular Frequency', suffixes=('_s', '_l'))
                    merged['Ratio'] = merged['Storage Modulus'] / merged['Loss Modulus']
                    all_ratios.append(merged[['Angular Frequency', 'Ratio']])
                
                combined = pd.concat(all_ratios)
                g = combined.groupby('Angular Frequency')['Ratio']
                mean_r, std_r = g.mean(), g.std()
                
                ax.plot(mean_r.index, mean_r.values, color=color, lw=3, label=dc)
                ax.fill_between(mean_r.index, mean_r - std_r, mean_r + std_r, color=color, alpha=0.2)
                
                legend_handles.append(color)
                legend_labels.append(dc)
            
            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.set_ylim(1, 1000)
            ax.set_xlabel("Angular Frequency (rad/s)", fontsize=AXIS_FS)
            ax.set_ylabel("G'/G'' Ratio", fontsize=AXIS_FS)
            ax.tick_params(axis='both', which='major', labelsize=TICK_FS, length=3, width=0.8)
            
            custom_lines = [Line2D([0], [0], color=color, lw=3) for color in legend_handles]
            small_legend(ax, custom_lines, legend_labels, title="Duty Cycle", 
                         bbox_to_anchor=(1, 1.02), loc='upper left',
                         handler_map={Line2D: HandlerLineWithShade()})
            
            plt.tight_layout()
            
            filename = f"gelatin_{gelatin_conc}_{particle}_ratio_line.png"
            filepath = os.path.join(output_dir, filename)
            plt.savefig(filepath, dpi=EXPORT_DPI, bbox_inches='tight')
            plt.close()
            plots_created.append(filename)
    
    # Save t-test results
    if t_test_results:
        t_test_file = os.path.join(output_dir, "all_t_test_results.txt")
        with open(t_test_file, 'w') as f:
            f.write("Statistical Analysis Results\n")
            f.write("="*60 + "\n")
            f.write("Note: Two-tailed tests detect any difference.\n")
            f.write("      One-tailed p-value = Two-tailed p-value / 2\n")
            f.write("      (assumes storage modulus decreases with duty cycle)\n")
            f.write("="*60 + "\n\n")
            
            f.write("INDEPENDENT T-TESTS (Duty Cycle Comparisons)\n")
            f.write("-"*60 + "\n\n")
            for gelatin_conc in sorted_gelatin:
                gelatin_tests = [r for r in t_test_results if r.startswith(f"{gelatin_conc} Gelatin")]
                if gelatin_tests:
                    f.write(f"{gelatin_conc} Gelatin:\n")
                    f.write("-"*30 + "\n")
                    for result in gelatin_tests:
                        f.write(result.replace(f"{gelatin_conc} Gelatin - ", "  ") + "\n")
                    f.write("\n")
        
        plots_created.append("all_t_test_results.txt")
        print(f"\n✓ Saved statistical results to: {t_test_file}")
    
    return output_dir, plots_created, file_data


def create_focus_vs_distance_comparisons(csv_files, global_ymax_storage, global_ymax_loss, base_directory):
    """Create bar charts comparing Focus vs Distance measurements for each gelatin/particle/duty cycle."""
    
    focus_vs_distance_dir = os.path.join(base_directory, "focus_vs_distance")
    os.makedirs(focus_vs_distance_dir, exist_ok=True)
    
    freq_window = (0.3, 1.0)
    
    cycle_map = {
        '0 cycles': '0%',
        '28 cycles': '2.5%',
        '40 cycles': '3.6%'
    }
    
    particle_colors = {
        'DBPC HMSN': '#D55E00',
        'MSN': '#009E73',
        '1X PBS': '#0072B2',
    }
    
    shade_map = {
        '0%': 1.3,
        '2.5%': 1.0,
        '3.6%': 0.7
    }
    
    # Organize files by gelatin, particle, duty cycle, and Focus/Distance
    file_data = {}
    
    for csv_path in csv_files:
        gelatin_conc = extract_gelatin_concentration(csv_path)
        if not gelatin_conc:
            continue
        
        filename = os.path.basename(csv_path)
        lower = filename.lower()
        
        cycle_type, dc = detect_cycle_in_filename(filename)
        if not (cycle_type and dc):
            continue
        
        if 'dbpc hmsn' in lower or 'dbpc_hmsn' in lower or 'dbpchmsn' in lower:
            particle = 'DBPC HMSN'
        elif 'msn' in lower and 'hmsn' not in lower:
            particle = 'MSN'
        elif '1x pbs' in lower or '1xpbs' in lower or 'pbs' in lower:
            particle = '1X PBS'
        else:
            continue
        
        focus_dist = detect_focus_distance_filename(filename)
        if not focus_dist:
            continue
        
        if gelatin_conc not in file_data:
            file_data[gelatin_conc] = {
                p: {dc: {'Focus': [], 'Distance': []} for dc in cycle_map.values()}
                for p in particle_colors
            }
        
        file_data[gelatin_conc][particle][dc][focus_dist].append(csv_path)
    
    sorted_gelatin = sorted(file_data.keys(), key=lambda x: int(x.rstrip('%')))
    plots_created = []
    paired_t_test_results = []
    
    print(f"\nCreating Focus vs Distance comparison charts...")
    print(f"Output directory: {os.path.relpath(focus_vs_distance_dir, base_directory)}\n")
    
    # Create comparison plots for each combination
    for gelatin_conc in sorted_gelatin:
        focus_pattern = GELATIN_PATTERNS[gelatin_conc]
        distance_pattern = GELATIN_PATTERNS[gelatin_conc] + '...'
        
        for particle in particle_colors:
            for modulus_type in ['storage', 'loss']:
                modulus_label = 'Storage' if modulus_type == 'storage' else 'Loss'
                modulus_symbol = "G'" if modulus_type == 'storage' else 'G"'
                global_ymax = global_ymax_storage if modulus_type == 'storage' else global_ymax_loss
                col_index = 6 if modulus_type == 'storage' else 7
                
                # Check which duty cycles have both Focus and Distance data
                available_dcs = []
                dc_focus_means = {}
                dc_focus_stds = {}
                dc_distance_means = {}
                dc_distance_stds = {}
                
                for dc in sorted(cycle_map.values(), key=lambda x: float(x.rstrip('%'))):
                    focus_files_dc = file_data[gelatin_conc][particle][dc]['Focus']
                    distance_files_dc = file_data[gelatin_conc][particle][dc]['Distance']
                    
                    if focus_files_dc and distance_files_dc:
                        available_dcs.append(dc)
                        
                        # Process Focus files
                        if focus_files_dc:
                            focus_vals = []
                            for f_path in focus_files_dc:
                                try:
                                    df = pd.read_csv(f_path, skiprows=53, header=None)
                                    data = df.iloc[:, [0, col_index]].dropna()
                                    data.columns = ['Angular Frequency', f'{modulus_label} Modulus']
                                    data = data.apply(pd.to_numeric, errors='coerce').dropna()
                                    
                                    mean_val = data[(data['Angular Frequency']>=freq_window[0]) &
                                                   (data['Angular Frequency']<=freq_window[1])][f'{modulus_label} Modulus'].mean()
                                    if not np.isnan(mean_val):
                                        focus_vals.append(mean_val)
                                except:
                                    pass
                            
                            dc_focus_means[dc] = np.mean(focus_vals) if focus_vals else 0
                            dc_focus_stds[dc] = np.std(focus_vals) if focus_vals else 0
                        else:
                            dc_focus_means[dc] = 0
                            dc_focus_stds[dc] = 0
                        
                        # Process Distance files
                        if distance_files_dc:
                            distance_vals = []
                            for d_path in distance_files_dc:
                                try:
                                    df = pd.read_csv(d_path, skiprows=53, header=None)
                                    data = df.iloc[:, [0, col_index]].dropna()
                                    data.columns = ['Angular Frequency', f'{modulus_label} Modulus']
                                    data = data.apply(pd.to_numeric, errors='coerce').dropna()
                                    
                                    mean_val = data[(data['Angular Frequency']>=freq_window[0]) &
                                                   (data['Angular Frequency']<=freq_window[1])][f'{modulus_label} Modulus'].mean()
                                    if not np.isnan(mean_val):
                                        distance_vals.append(mean_val)
                                except:
                                    pass
                            
                            dc_distance_means[dc] = np.mean(distance_vals) if distance_vals else 0
                            dc_distance_stds[dc] = np.std(distance_vals) if distance_vals else 0
                        else:
                            dc_distance_means[dc] = 0
                            dc_distance_stds[dc] = 0
                        
                        # Paired t-test between Focus and Distance
                        if focus_vals and distance_vals and len(focus_vals) == len(distance_vals) and len(focus_vals) >= 2:
                            try:
                                t_stat, p_val_two = ttest_rel(focus_vals, distance_vals)
                                p_val_one = p_val_two / 2
                                
                                sig_two = "***" if p_val_two < 0.001 else "**" if p_val_two < 0.01 else "*" if p_val_two < 0.05 else "ns"
                                sig_one = "***" if p_val_one < 0.001 else "**" if p_val_one < 0.01 else "*" if p_val_one < 0.05 else "ns"
                                
                                result_str = (f"{gelatin_conc} Gelatin - {particle} {modulus_label} @ {dc}: "
                                            f"Focus vs Distance | "
                                            f"Two-tailed: p={p_val_two:.4f} {sig_two}, "
                                            f"One-tailed: p={p_val_one:.4f} {sig_one} | "
                                            f"n={len(focus_vals)}")
                                paired_t_test_results.append(result_str)
                            except:
                                pass
                
                if not available_dcs:
                    continue
                
                # Create bar chart
                fig, ax = small_panel_ax()
                
                x = np.arange(len(available_dcs))
                width = 0.35
                
                base_color = particle_colors[particle]
                
                # Focus bars (lighter shade)
                focus_means = [dc_focus_means[dc] for dc in available_dcs]
                focus_stds = [dc_focus_stds[dc] for dc in available_dcs]
                focus_bars = ax.bar(x - width/2, focus_means, width, 
                                   yerr=focus_stds, label='Focus', error_kw=ERR_KW)
                
                # Distance bars (darker shade)
                distance_means = [dc_distance_means[dc] for dc in available_dcs]
                distance_stds = [dc_distance_stds[dc] for dc in available_dcs]
                distance_bars = ax.bar(x + width/2, distance_means, width,
                                      yerr=distance_stds, label='Distance', error_kw=ERR_KW)
                
                # Color and pattern bars using shade_map for each duty cycle
                for i, dc in enumerate(available_dcs):
                    # Get base shade for this duty cycle
                    base_shade = shade_map[dc]
                    duty_cycle_color = adjust_lightness(base_color, base_shade)
                    
                    # Focus: gelatin pattern only
                    focus_bars[i].set_color(duty_cycle_color)
                    focus_bars[i].set_hatch(focus_pattern)
                    focus_bars[i].set_edgecolor('gray')
                    focus_bars[i].set_linewidth(0.5)
                    
                    # Distance: gelatin pattern + dots
                    distance_bars[i].set_color(duty_cycle_color)
                    distance_bars[i].set_hatch(distance_pattern)
                    distance_bars[i].set_edgecolor('gray')
                    distance_bars[i].set_linewidth(0.5)
                
                ax.set_xlabel('Duty Cycle (%)', fontsize=AXIS_FS)
                ax.set_ylabel(f'{modulus_label} Modulus ({modulus_symbol})\n(Pa)', fontsize=AXIS_FS)
                
                ax.set_xticks(x)
                ax.set_xticklabels([dc.rstrip('%') for dc in available_dcs], fontsize=TICK_FS)
                
                ax.set_ylim(0, global_ymax * 1.1)
                ax.ticklabel_format(style='plain', axis='y', useOffset=False)
                ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x):,}'))
                
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                
                # Legend - use middle duty cycle shade (2.5% = 1.0) for representation
                legend_color = adjust_lightness(base_color, 1.0)
                with mpl.rc_context({'hatch.linewidth': HATCH_LW_LEG}):
                    handles = [
                        Patch(facecolor=legend_color, 
                              edgecolor='gray', linewidth=0.5, hatch=focus_pattern),
                        Patch(facecolor=legend_color, 
                              edgecolor='gray', linewidth=0.5, hatch=distance_pattern)
                    ]
                small_legend(ax, handles=handles, labels=['Bottom layer (0-4 mm)', 'Top layer (4-8 mm)'],
                            title="Measurement Type", 
                            bbox_to_anchor=(1.00, 1), loc='upper left')
                
                plt.tight_layout()
                
                # Save to subfolder
                filename = f"focus_vs_distance_{gelatin_conc}_{particle.replace(' ', '_')}_{modulus_type}.png"
                filepath = os.path.join(focus_vs_distance_dir, filename)
                plt.savefig(filepath, dpi=EXPORT_DPI, bbox_inches='tight')
                plt.close()
                plots_created.append(filename)
                
                print(f"  Created: {filename}")
    
    # Save paired t-test results
    if paired_t_test_results:
        t_test_file = os.path.join(focus_vs_distance_dir, "focus_vs_distance_paired_t_tests.txt")
        with open(t_test_file, 'w') as f:
            f.write("PAIRED T-TESTS: Bottom Layer (0-4 mm) vs Top Layer (4-8 mm)\n")
            f.write("="*60 + "\n")
            f.write("Note: Two-tailed tests detect any difference.\n")
            f.write("      One-tailed p-value = Two-tailed p-value / 2\n")
            f.write("="*60 + "\n\n")
            
            for gelatin_conc in sorted_gelatin:
                gelatin_tests = [r for r in paired_t_test_results if r.startswith(f"{gelatin_conc} Gelatin")]
                if gelatin_tests:
                    f.write(f"{gelatin_conc} Gelatin:\n")
                    f.write("-"*30 + "\n")
                    for result in gelatin_tests:
                        f.write(result.replace(f"{gelatin_conc} Gelatin - ", "  ") + "\n")
                    f.write("\n")
        
        plots_created.append("focus_vs_distance_paired_t_tests.txt")
        print(f"\nSaved paired t-test results to: focus_vs_distance_paired_t_tests.txt")
    
    print(f"\nCompleted Focus vs Distance comparison charts")
    print(f"Saved to: {os.path.relpath(focus_vs_distance_dir, base_directory)}")
    
    return focus_vs_distance_dir, plots_created


# ===== MAIN EXECUTION =====
# Note: You need to define csv_files and global_ymax before running this section

# Find common base directory for all outputs
base_directory = find_common_base_directory(csv_files)
print(f"\nBase directory for outputs: {base_directory}")

# Calculate Loss modulus global y-max
global_ymax_loss = 0
for csv_path in csv_files:
    try:
        df = pd.read_csv(csv_path, skiprows=53, header=None)
        loss_data = df.iloc[0:, [0, 7]].dropna()
        loss_data.columns = ['Angular Frequency', 'Loss Modulus']
        loss_data = loss_data.apply(pd.to_numeric, errors='coerce').dropna()
        
        if not loss_data.empty:
            freq_mask = (loss_data['Angular Frequency'] >= 0.3) & (loss_data['Angular Frequency'] <= 1.0)
            if freq_mask.any():
                max_val = loss_data.loc[freq_mask, 'Loss Modulus'].mean()
                global_ymax_loss = max(global_ymax_loss, max_val)
    except:
        pass

print(f"\nGlobal Y-max Storage Modulus = {global_ymax:.1f} Pa")
print(f"Global Y-max Loss Modulus = {global_ymax_loss:.1f} Pa")

# Check if we have Focus/Distance files
focus_files = [f for f in csv_files if detect_focus_distance_filename(f) == 'Focus']
distance_files = [f for f in csv_files if detect_focus_distance_filename(f) == 'Distance']

if focus_files or distance_files:
    print(f"\nDetected Focus/Distance naming convention")
    print(f"Focus files: {len(focus_files)}, Distance files: {len(distance_files)}")
    print("Creating separate plots for Focus and Distance measurements\n")
    
    if focus_files:
        print("\n" + "="*60)
        print("CREATING FOCUS PLOTS")
        print("="*60 + "\n")
        fd_output_dir, fd_plots, _ = plot_gelatin_organized_data(focus_files, global_ymax, global_ymax_loss, 
                                                               base_directory, add_dots=False, output_suffix="_focus")
        print(f"\nCreated {len(fd_plots)} Focus plots in: {os.path.relpath(fd_output_dir, base_directory)}")
    
    if distance_files:
        print("\n" + "="*60)
        print("CREATING DISTANCE PLOTS (with dot pattern)")
        print("="*60 + "\n")
        dist_output_dir, dist_plots, _ = plot_gelatin_organized_data(distance_files, global_ymax, global_ymax_loss,
                                                                   base_directory, add_dots=True, output_suffix="_distance")
        print(f"\nCreated {len(dist_plots)} Distance plots with dot pattern in: {os.path.relpath(dist_output_dir, base_directory)}")
    
    # ===== Create Focus vs Distance Comparison Charts =====
    print("\n" + "="*60)
    print("CREATING FOCUS VS DISTANCE COMPARISON CHARTS")
    print("="*60 + "\n")
    fvd_dir, fvd_plots = create_focus_vs_distance_comparisons(csv_files, global_ymax, global_ymax_loss, base_directory)
    
else:
    print("\nNo Focus/Distance naming detected - processing all files together\n")
    print("="*60)
    print("CREATING PLOTS FOR ALL DATA")
    print("="*60 + "\n")
    
    output_dir, plots, _ = plot_gelatin_organized_data(csv_files, global_ymax, global_ymax_loss, 
                                                     base_directory, add_dots=False, output_suffix="")
    print(f"\nCreated {len(plots)} plots in: {os.path.relpath(output_dir, base_directory)}")

print("\n" + "="*60)
print("BATCH PROCESSING COMPLETE!")
print(f"Total CSV files processed: {len(csv_files)}")
if focus_files or distance_files:
    print(f"Focus vs Distance comparison plots created: {len(fvd_plots)}")
print("="*60)
print("\n✓ Check the output folders for plots and statistical results!")
print("✓ Statistical test results saved in text files")
print("\n" + "="*60)
print("OUTPUT FOLDER STRUCTURE:")
print("="*60)
print(f"📁 {base_directory}/")
print("   ├── 📁 plots_by_gelatin_focus/")
print("   ├── 📁 plots_by_gelatin_distance/")
print("   └── 📁 focus_vs_distance/")
print("="*60)


Base directory for outputs: C:\Users\INSTRU~1\ONEDRI~2\Courses\YEAR1~1\FALLSE~1\CHEN58~1\HYDROG~1\RHEOME~1\FALLSE~1

Global Y-max Storage Modulus = 2769.0 Pa
Global Y-max Loss Modulus = 398.8 Pa

Detected Focus/Distance naming convention
Focus files: 54, Distance files: 54
Creating separate plots for Focus and Distance measurements


CREATING FOCUS PLOTS


✓ Saved statistical results to: C:\Users\INSTRU~1\ONEDRI~2\Courses\YEAR1~1\FALLSE~1\CHEN58~1\HYDROG~1\RHEOME~1\FALLSE~1\plots_by_gelatin_focus\all_t_test_results.txt

Created 19 Focus plots in: plots_by_gelatin_focus

CREATING DISTANCE PLOTS (with dot pattern)


✓ Saved statistical results to: C:\Users\INSTRU~1\ONEDRI~2\Courses\YEAR1~1\FALLSE~1\CHEN58~1\HYDROG~1\RHEOME~1\FALLSE~1\plots_by_gelatin_distance\all_t_test_results.txt

Created 19 Distance plots with dot pattern in: plots_by_gelatin_distance

CREATING FOCUS VS DISTANCE COMPARISON CHARTS


Creating Focus vs Distance comparison charts...
Output directory: focus_vs_distance

 

In [ ]:
# === List All Sheet Names in Excel Files ===
# Run this after Block 2 to see all sheet names in your Excel files

import pandas as pd
import os

print("="*60)
print("EXCEL FILE SHEET NAMES")
print("="*60)

for i, excel_file in enumerate(excel_files, 1):
    rel_path = os.path.relpath(excel_file, base_directory)
    print(f"\n{i}. {rel_path}")
    print("-" * len(f"{i}. {rel_path}"))
    
    try:
        xls = pd.ExcelFile(excel_file)
        sheet_names = xls.sheet_names
        
        for j, sheet in enumerate(sheet_names, 1):
            print(f"   Sheet {j}: '{sheet}'")
            
            # Show what the code detects in this sheet name
            lower = sheet.lower()
            detections = []
            
            # Check for cycles
            if '0 cycles' in lower:
                detections.append("0 cycles (0%)")
            if '40 cycles' in lower:
                detections.append("40 cycles (3.6%)")
                
            # Check for particles
            if 'dbpc hmsn' in lower or 'dbpc_hmsn' in lower:
                detections.append("DBPC HMSN")
            elif 'msn' in lower and 'hmsn' not in lower:
                detections.append("MSN")
            elif '1x pbs' in lower or '1xpbs' in lower or 'pbs' in lower:
                detections.append("PBS")
                
            if detections:
                print(f"      → Detected: {', '.join(detections)}")
            else:
                print(f"      → No matches (will be skipped)")
                
    except Exception as e:
        print(f"   ERROR reading file: {str(e)}")

print("\n" + "="*60)
print(f"Total Excel files examined: {len(excel_files)}")
print("="*60)